<a href="https://colab.research.google.com/github/jonruida/PFM-and-PEFM-for-Image-Anomaly-Detection-and-Segmentation/blob/main/Transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import zipfile
import os

# URL del archivo en Dropbox (modificada para descarga directa)
dropbox_url = "https://www.dropbox.com/scl/fi/7vd2tqg5tduieq0bk7k6f/ZgZ_s3.zip?rlkey=70iefk1dtwva3py1fbrz444h7&st=mvqw6fpd&dl=1"


In [2]:

# Ruta donde se guardará el archivo descargado
zip_path = '/content/ZgZ_s3.zip'

# Ruta donde se extraerá el contenido del ZIP
extract_path = "/content/data"

# Crear el directorio de extracción si no existe
os.makedirs(extract_path, exist_ok=True)

# Descargar el archivo desde Dropbox
try:
    response = requests.get(dropbox_url)
    response.raise_for_status()  # Verificar si la descarga fue exitosa
    with open(zip_path, 'wb') as file:
        file.write(response.content)
    print("Descarga exitosa.")
except requests.exceptions.RequestException as e:
    print(f"Error al descargar el archivo: {e}")
    raise

# Verificar si el archivo descargado es un archivo ZIP válido y extraerlo
if zipfile.is_zipfile(zip_path):
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print("Extracción exitosa.")
    except zipfile.BadZipFile:
        print("Error: El archivo no es un archivo ZIP válido o está corrupto.")
    except Exception as e:
        print(f"Error inesperado durante la extracción: {e}")
else:
    print("Error: El archivo descargado no es un archivo ZIP válido.")


Descarga exitosa.
Extracción exitosa.


Convertir a PNG

In [3]:
from PIL import Image
import os
import glob

def convertir_a_png(imagen_path, output_dir):
    imagen = Image.open(imagen_path)
    base_nombre = os.path.basename(imagen_path).split(".")[0]
    output_path = os.path.join(output_dir, f"{base_nombre}.png")
    imagen.save(output_path, "PNG")
    return output_path

# Directorios de entrada y salida
malignas_path = '/content/data/Malignos--Output_S3/imgS3_rois'
benignas_path = '/content/data/PAPANICOL--Output_S3/imgS3_rois'
output_dir_malignas = '/content/data/Malignos--Output_S3/imgS3_rois_png'
output_dir_benignas = '/content/data/PAPANICOL--Output_S3/imgS3_rois_png'

# Crear los directorios de salida si no existen
os.makedirs(output_dir_malignas, exist_ok=True)
os.makedirs(output_dir_benignas, exist_ok=True)

# Convertir imágenes malignas a PNG
for img_path in glob.glob(os.path.join(malignas_path, '*.tif')):
    convertir_a_png(img_path, output_dir_malignas)

# Convertir imágenes benignas a PNG
for img_path in glob.glob(os.path.join(benignas_path, '*.tif')):
    convertir_a_png(img_path, output_dir_benignas)



Trocear imagenes

In [4]:
import os
import random
import shutil
from PIL import Image
import glob

# Directorios de origen
dir_sin_anomalias = '/content/data/PAPANICOL--Output_S3/imgS3_rois_png'
dir_con_anomalias = '/content/data/Malignos--Output_S3/imgS3_rois_png'

# Directorios de destino
dir_test = '/content/data/test'
dir_train = '/content/data/train'

# Crear directorios de test y train si no existen
os.makedirs(dir_test, exist_ok=True)
os.makedirs(dir_train, exist_ok=True)

# Listar imágenes sin anomalías
imagenes_sin_anomalias = [
    "10.png", "19.png", "27.png", "35.png", "43.png", "51.png", "5.png", "68.png", "76.png", "84.png", "92.png",
    "11.png", "1.png", "28.png", "36.png", "44.png", "52.png", "60.png", "69.png", "77.png", "85.png", "93.png",
    "12.png", "20.png", "29.png", "37.png", "45.png", "53.png", "61.png", "6.png", "78.png", "86.png", "94.png",
    "13.png", "21.png", "2.png", "38.png", "46.png", "54.png", "62.png", "70.png", "79.png", "87.png", "95.png",
    "14.png", "22.png", "30.png", "39.png", "47.png", "55.png", "63.png", "71.png", "7.png", "88.png", "96.png",
    "15.png", "23.png", "31.png", "3.png", "48.png", "56.png", "64.png", "72.png", "80.png", "89.png", "97.png",
    "16.png", "24.png", "32.png", "40.png", "49.png", "57.png", "65.png", "73.png", "81.png", "8.png", "98.png",
    "17.png", "25.png", "33.png", "41.png", "4.png", "58.png", "66.png", "74.png", "82.png", "90.png", "9.png",
    "18.png", "26.png", "34.png", "42.png", "50.png", "59.png", "67.png", "75.png", "83.png", "91.png"
]

# Listar imágenes con anomalías
imagenes_con_anomalias = [
    "1005124.png", "1005129.png", "1005133.png", "1005207.png", "1005211.png", "1005215.png",
    "1005126.png", "1005130.png", "1005134.png", "1005208.png", "1005212.png",
    "1005127.png", "1005131.png", "1005205.png", "1005209.png", "1005213.png",
    "1005128.png", "1005132.png", "1005206.png", "1005210.png", "1005214.png"
]

# Seleccionar aleatoriamente 21 imágenes sin anomalías para el test
imagenes_sin_anomalias_test = random.sample(imagenes_sin_anomalias, 21)
# Las restantes serán para el entrenamiento
imagenes_sin_anomalias_train = list(set(imagenes_sin_anomalias) - set(imagenes_sin_anomalias_test))

# Copiar imágenes sin anomalías a los directorios correspondientes
for img in imagenes_sin_anomalias_test:
    shutil.copy(os.path.join(dir_sin_anomalias, img), os.path.join(dir_test, img))

for img in imagenes_sin_anomalias_train:
    shutil.copy(os.path.join(dir_sin_anomalias, img), os.path.join(dir_train, img))

# Copiar imágenes con anomalías al directorio de test
for img in imagenes_con_anomalias:
    shutil.copy(os.path.join(dir_con_anomalias, img), os.path.join(dir_test, img))

# Función para trocear una imagen en N partes
def trocear_imagen(imagen_path, filas, columnas, output_dir):
    imagen = Image.open(imagen_path)
    anchura, altura = imagen.size

    altura_trozo = altura // filas
    anchura_trozo = anchura // columnas

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i in range(filas):
        for j in range(columnas):
            trozo = imagen.crop((j * anchura_trozo, i * altura_trozo, (j + 1) * anchura_trozo, (i + 1) * altura_trozo))
            trozo.save(os.path.join(output_dir, f'{os.path.basename(imagen_path).split(".")[0]}_{i}_{j}.png'))

# Parámetros para trocear
filas = 34
columnas = 34

# Trocear imágenes del conjunto de entrenamiento
output_dir_train_troceadas = os.path.join(dir_train, 'good')
for img_path in glob.glob(os.path.join(dir_train, '*.png')):
    trocear_imagen(img_path, filas, columnas, output_dir_train_troceadas)

# Trocear imágenes del conjunto de test
output_dir_test_troceadas = os.path.join(dir_test, 'good')
for img_path in glob.glob(os.path.join(dir_test, '*.png')):
    trocear_imagen(img_path, filas, columnas, output_dir_test_troceadas)


In [5]:
!git clone https://github.com/jonruida/PFM-and-PEFM-for-Image-Anomaly-Detection-and-Segmentation

Cloning into 'PFM-and-PEFM-for-Image-Anomaly-Detection-and-Segmentation'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 113 (delta 57), reused 78 (delta 28), pack-reused 0
Receiving objects: 100% (113/113), 4.13 MiB | 26.92 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [6]:
!pip install loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.1 MB/s eta 0:00:00


In [11]:
!pip install torch


In [7]:
!pip install thop


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [15]:
!python /content/PFM-and-PEFM-for-Image-Anomaly-Detection-and-Segmentation/MB-PFM-ResNet.py --train --gpu_id 0 --batch_size 8 --epochs 200 --resize 256 --data_trans imagenet --loss_type l2norm+l2 --data_root /content/data/


2024-06-04 17:35:06.274805: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 17:35:06.274867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 17:35:06.276367: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-04 17:35:06.284400: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 17:35:07.680162: W tensorflow/compiler/tf2